<a href="https://colab.research.google.com/github/Dohy-Lee/NLP_By_Pytorch/blob/main/ch6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

시퀀스 : 순서가 있는 항목의   
순차데이터(Sequence Data) : 언어, 음성, 시계열처럼 한 데이터 항목이 앞뒤 항목에 의존하는 데이터  
은닉 상태 : 지금까지 시퀀스에서 본 모든 정보를 담음  
RNN의 목적은 주어진 은닉 상태 벡터와 입력 벡터에 대한 출력을 계산함으로써 시퀀스 불변성을 학습  
RNN은 현재 입력 정보와 지금까지 본 입력을 요약한 상태 표현을 사용하는 방법을 배움  
RNN은 어느 타임 스텝에 있는지 알 방법이 없지만, 한 타임 스텝에서 다른 타임 스텝으로 이동하면서 손실 함수를 최소화하기 위해 상태 표현을 관리하는 방법을 학습  


# 엘만 RNN 구현하기

In [1]:
from argparse import Namespace
import os
import json

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import tqdm

In [2]:
class Vocabulary(object): # 매핑을 위해 텍스트를 처리하고 어휘 사전을 만드는 클래스
    def __init__(self, token_to_idx=None): # token_to_idx (dict) : 기존 토큰-인덱스 매핑 딕셔너리
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx

        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
        
    def to_serializable(self): # 직렬화할 수 있는 딕셔너리를 반환
        return {'token_to_idx': self._token_to_idx}

    @classmethod
    def from_serializable(cls, contents): # 직렬화된 딕셔너리에서 Vocabulary 객체 생성
        return cls(**contents)

    def add_token(self, token): # 토큰을 기반으로 매핑 딕셔너리 업데이트
                                # token (str)  : Vocabulary에 추가할 토큰
                                # 반환값 : index(int) : 토큰에 상응하는 정수
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
            
    def add_many(self, tokens): # 토큰 리스트를 Vocabulary에 추가
                                # tokens(list) : 문자열 토큰 리스트
                                # indices(list) : 토큰 리스트에 상응되는 인덱스 리스트
        return [self.add_token(token) for token in tokens]

    def lookup_token(self, token): # 토큰에 대응하는 인덱스 추출
                                   # token (str) : 찾을 토큰
                                   # 반환값 : index (int) : 해당 토큰의 인덱스
        return self._token_to_idx[token]

    def lookup_index(self, index): # 인덱스에 해당하는 토큰 반환
                                   # index (int) : 찾을 토큰의 인덱스
                                   # 반환값 : token (str) : 해당 인덱스의 토큰
        if index not in self._idx_to_token: # 인덱스가 Vocabulary에 없을 때 발생
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self._token_to_idx)

In [3]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_seq_token = begin_seq_token
        self._end_seq_token = end_seq_token

        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)
        self.begin_seq_index = self.add_token(self._begin_seq_token)
        self.end_seq_index = self.add_token(self._end_seq_token)

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self._unk_token,
                         'mask_token': self._mask_token,
                         'begin_seq_token': self._begin_seq_token,
                         'end_seq_token': self._end_seq_token})
        return contents

    def lookup_token(self, token): # 토큰에 대응하는 인덱스 추출. 토큰이 없으면 UNK 인덱스 반환
                                   # UNK 토큰을 사용하려면 (Vocabulary에 추가하기 위해) 'unk_index'가 0보다 커야함
                                   # token (str) : 찾을 토큰
                                   # 반환값 : index (int) : 해당 토큰의 인덱스 
        if self.unk_index >= 0:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

In [4]:
class SurnameVectorizer(object): # 어휘 사전을 생성하고 관리
    def __init__(self, char_vocab, nationality_vocab): # char_vocab (Vocabulary) : 문자를 정수로 매핑
                                                       # nationality_vocab (Vocabulary) : 국적을 정수로 매핑
        self.char_vocab = char_vocab
        self.nationality_vocab = nationality_vocab

    def vectorize(self, surname, vector_length=-1): # surname (str) : 문자열
                                                    # vector_length (int) : 인덱스 벡터의 길이를 맞추기 위한 매개 변수
        indices = [self.char_vocab.begin_seq_index]
        indices.extend(self.char_vocab.lookup_token(token) 
                       for token in surname)
        indices.append(self.char_vocab.end_seq_index)

        if vector_length < 0:
            vector_length = len(indices)

        out_vector = np.zeros(vector_length, dtype=np.int64)         
        out_vector[:len(indices)] = indices
        out_vector[len(indices):] = self.char_vocab.mask_index
        
        return out_vector, len(indices)

    @classmethod
    def from_dataframe(cls, surname_df): # 데이터셋 데이터프레임으로 SurnameVectorizer 객체를 초기화
                                         # surname_df (pandas.DataFrame) : 성씨 데이터 셋
                                         # 반환값 : SuranmeVectorizer 객체
        char_vocab = SequenceVocabulary()
        nationality_vocab = Vocabulary()

        for index, row in surname_df.iterrows():
            for char in row.surname:
                char_vocab.add_token(char)
            nationality_vocab.add_token(row.nationality)

        return cls(char_vocab, nationality_vocab)

    @classmethod
    def from_serializable(cls, contents):
        char_vocab = SequenceVocabulary.from_serializable(contents['char_vocab'])
        nat_vocab =  Vocabulary.from_serializable(contents['nationality_vocab'])

        return cls(char_vocab=char_vocab, nationality_vocab=nat_vocab)

    def to_serializable(self):
        return {'char_vocab': self.char_vocab.to_serializable(), 
                'nationality_vocab': self.nationality_vocab.to_serializable()}

In [5]:
class SurnameDataset(Dataset):
    def __init__(self, surname_df, vectorizer): # surname_df (pandas.DataFrame): 데이터셋
                                                # vectorizer (SurnameVectorizer): 데이터셋에서 만든 Vectorizer 객체
        self.surname_df = surname_df 
        self._vectorizer = vectorizer

        self._max_seq_length = max(map(len, self.surname_df.surname)) + 2

        self.train_df = self.surname_df[self.surname_df.split=='train']
        self.train_size = len(self.train_df)

        self.val_df = self.surname_df[self.surname_df.split=='val']
        self.validation_size = len(self.val_df)

        self.test_df = self.surname_df[self.surname_df.split=='test']
        self.test_size = len(self.test_df)

        self._lookup_dict = {'train': (self.train_df, self.train_size), 
                             'val': (self.val_df, self.validation_size), 
                             'test': (self.test_df, self.test_size)}

        self.set_split('train')
        
        # 클래스 가중치
        class_counts = self.train_df.nationality.value_counts().to_dict()
        def sort_key(item):
            return self._vectorizer.nationality_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

        
    @classmethod
    def load_dataset_and_make_vectorizer(cls, surname_csv): # 데이터셋을 로드하고 새로운 Vectorizer 객체 생성
                                                            # suranme_csv (str): 데이터셋의 위치
                                                            # 반환값: SurnameDataset의 객체
        surname_df = pd.read_csv(surname_csv)
        train_surname_df = surname_df[surname_df.split=='train']
        return cls(surname_df, SurnameVectorizer.from_dataframe(train_surname_df))
        
    @classmethod
    def load_dataset_and_load_vectorizer(cls, surname_csv, vectorizer_filepath): # 데이터셋과 새로운 Vectorizer 객체를 로드. 캐시된 Vectorizer 객체를 재사용
                                                                                 # surname_csv (str): 데이터셋의 위치
                                                                                 # vectorizer_filepath (str): Vectorizer 객체의 저장 위치
                                                                                 # 반환값 : SurnameDataset의 인스턴스
        surname_df = pd.read_csv(surname_csv)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(surname_df, vectorizer)

    @staticmethod
    def load_vectorizer_only(vectorizer_filepath): # 파일에서 Vectorizer 객체를 로드하는 정적 메서드
                                                   # vectorizer_filepath(str) : 직렬화된 Vectorzier 객체의 위치
                                                   # 반환값 : SurnameVectorizer의 인스턴스
        with open(vectorizer_filepath) as fp:
            return SurnameVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath): # Vectorizer 객체를 json 형태로 저장
                                                    # vectorizer_filepath(str) : Vectorizer 객체의 저장 위치
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self._vectorizer.to_serializable(), fp)

    def get_vectorizer(self): # 벡터 변환 객체 반환
        return self._vectorizer

    def set_split(self, split="train"):
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index): # 파이토치 데이터셋의 주요 진입 메서드
                                  # index (int): 데이터 포인트 인덱스
                                  # 반환값 : 다음 값을 담고 있는 딕셔너리 - 특성 (x_data), 레이블 (y_target), 특성 길이 (x_length)
        row = self._target_df.iloc[index]
        
        surname_vector, vec_length = \
            self._vectorizer.vectorize(row.surname, self._max_seq_length)
        
        nationality_index = \
            self._vectorizer.nationality_vocab.lookup_token(row.nationality)

        return {'x_data': surname_vector, 
                'y_target': nationality_index, 
                'x_length': vec_length}

    def get_num_batches(self, batch_size): # 배치 크기가 주어지면 데이터셋으로 만들 수 있는 배치 개수 반환
                                           # batch_size (int)
                                           # 반환값 : 배치 개수
        return len(self) // batch_size

    

def generate_batches(dataset, batch_size, shuffle=True, # 파이토치 DataLoader를 감싸고 있는 제너레이터 함수. 각 텐서를 지정된 장치로 이동시킴
                     drop_last=True, device="cpu"): 
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict

In [6]:
def column_gather(y_out, x_lengths): # 배치 행 인덱스를 순회하면서, x_lengths에 있는 값에 해당하는 인덱스 위치의 벡터를 반환(y_out에 있는 각 데이터 포인트에서 마지막 벡터를 추출)
                                     # y_out (torch.FloatTensor, torch.cuda.FloatTensor), shape: (batch, sequence, feature)
                                     # x_lengths (torch.LongTensor, torch.cuda.LongTensor), shape: (batch,) 
                                     # 반환값 : y_out (torch.FloatTensor, torch.cuda.FloatTensor), shape: (batch, feature)
    x_lengths = x_lengths.long().detach().cpu().numpy() - 1

    out = []
    for batch_index, column_index in enumerate(x_lengths):
        out.append(y_out[batch_index, column_index])

    return torch.stack(out)


class ElmanRNN(nn.Module): # RNNCell을 사용하여 구현한 엘만 RNN
    def __init__(self, input_size, hidden_size, batch_first=False): # input_size (int): 입력 벡터 크기
                                                                    # hidden_size (int): 은닉 상태 벡터 크기
                                                                    # batch_first (bool): 0번째 차원이 배치인지 여부. True로 설정하면 RNN이 입력 텐서의 0번째와 1번째 차원을 바꿈

        super(ElmanRNN, self).__init__()
        
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)
        
        self.batch_first = batch_first
        self.hidden_size = hidden_size

    def _initial_hidden(self, batch_size):
        return torch.zeros((batch_size, self.hidden_size))

    def forward(self, x_in, initial_hidden=None): # ElmanRNN의 정방향 계산
                                                  # x_in (torch.Tensor): 입력 데이터 텐서 
                                                  # 0번째 차원이 배치라면 x_in.shape = (batch_size, seq_size, feat_size), 아니라면 x_in.shape = (seq_size, batch_size, feat_size)
                                                  # initial_hidden (torch.Tensor): RNN의 초기 은닉 상태
                                                  # 반환값 : hiddens (torch.Tensor): 각 타임 스텝에서 RNN 출력
                                                  # 0번째 차원이 배치라면 hiddens.shape = (batch_size, seq_size, hidden_size)
                                                  # hiddens.shape = (seq_size, batch_size, hidden_size)
        if self.batch_first:
            batch_size, seq_size, feat_size = x_in.size()
            x_in = x_in.permute(1, 0, 2)
        else:
            seq_size, batch_size, feat_size = x_in.size()
    
        hiddens = []

        if initial_hidden is None:
            initial_hidden = self._initial_hidden(batch_size)
            initial_hidden = initial_hidden.to(x_in.device)

        hidden_t = initial_hidden
                    
        for t in range(seq_size):
            hidden_t = self.rnn_cell(x_in[t], hidden_t)
            hiddens.append(hidden_t)
            
        hiddens = torch.stack(hiddens)

        if self.batch_first:
            hiddens = hiddens.permute(1, 0, 2)

        return hiddens


class SurnameClassifier(nn.Module): # RNN으로 특성을 추출하고 MLP로 분류하는 모델
    def __init__(self, embedding_size, num_embeddings, num_classes, # embedding_size (int): 문자 임베딩의 크기
                 rnn_hidden_size, batch_first=True, padding_idx=0): # num_embeddings (int): 임베딩할 문자 개수
                                                                    # num_classes (int): 예측 벡터의 크기 (국적의 개수)
                                                                    # rnn_hidden_size (int): RNN의 은닉 상태 크기
                                                                    # batch_first (bool): 입력 텐서의 0번째 차원이 배치인지 시퀀스인지 나타내는 플래그
                                                                    # padding_idx (int): 텐서 패딩을 위한 인덱스
        super(SurnameClassifier, self).__init__()

        self.emb = nn.Embedding(num_embeddings=num_embeddings,
                                embedding_dim=embedding_size,
                                padding_idx=padding_idx)
        self.rnn = ElmanRNN(input_size=embedding_size,
                             hidden_size=rnn_hidden_size,
                             batch_first=batch_first)
        self.fc1 = nn.Linear(in_features=rnn_hidden_size,
                         out_features=rnn_hidden_size)
        self.fc2 = nn.Linear(in_features=rnn_hidden_size,
                          out_features=num_classes)

    def forward(self, x_in, x_lengths=None, apply_softmax=False): # 분류기의 정방향 계산
                                                                  # x_in (torch.Tensor): 입력 데이터 텐서, x_in.shape는 (batch, input_dim)
                                                                  # x_lengths (torch.Tensor): 배치에 있는 각 시퀀스의 길이. 시퀀스의 마지막 벡터를 찾는데 사용
                                                                  # apply_softmax (bool): 소프트맥스 활성화 함수를 위한 플래그. 크로스-엔트로피 손실을 사용하려면 False로 지정
                                                                  # 반환값 : 결과 텐서. tensor.shape는 (batch, output_dim)
        x_embedded = self.emb(x_in)
        y_out = self.rnn(x_embedded)

        if x_lengths is not None:
            y_out = column_gather(y_out, x_lengths)
        else:
            y_out = y_out[:, -1, :]

        y_out = F.relu(self.fc1(F.dropout(y_out, 0.5)))
        y_out = self.fc2(F.dropout(y_out, 0.5))

        if apply_softmax:
            y_out = F.softmax(y_out, dim=1)

        return y_out

In [7]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [8]:
args = Namespace(
    # 경로 정보
    surname_csv="data/surnames/surnames_with_splits.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="model_storage/ch6/surname_classification",
    # 모델 하이퍼파라미터
    char_embedding_size=100,
    rnn_hidden_size=64,
    # 훈련 하이퍼파라미터
    num_epochs=100,
    learning_rate=1e-3,
    batch_size=64,
    seed=1337,
    early_stopping_criteria=5,
    # 실행 옵션
    cuda=True,
    catch_keyboard_interrupt=True,
    reload_from_files=False,
    expand_filepaths_to_save_dir=True,
)

# CUDA 체크
if not torch.cuda.is_available():
    args.cuda = False

args.device = torch.device("cuda" if args.cuda else "cpu")
    
print("CUDA 사용여부: {}".format(args.cuda))


if args.expand_filepaths_to_save_dir:
    args.vectorizer_file = os.path.join(args.save_dir,
                                        args.vectorizer_file)

    args.model_state_file = os.path.join(args.save_dir,
                                         args.model_state_file)
    
# 시드 설정
set_seed_everywhere(args.seed, args.cuda)

# 디렉토리 처리
handle_dirs(args.save_dir)

CUDA 사용여부: True


In [9]:
!mkdir data
!wget https://git.io/JtSPf -O data/download.py
!wget https://git.io/JtSPU -O data/get-all-data.sh
!chmod 755 data/get-all-data.sh
%cd data
!./get-all-data.sh
%cd ..

--2022-09-27 07:04:40--  https://git.io/JtSPf
Resolving git.io (git.io)... 140.82.114.22
Connecting to git.io (git.io)|140.82.114.22|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_6/classifying-surnames/data/download.py [following]
--2022-09-27 07:04:40--  https://raw.githubusercontent.com/rickiepark/nlp-with-pytorch/main/chapter_6/classifying-surnames/data/download.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1572 (1.5K) [text/plain]
Saving to: ‘data/download.py’

data/download.py    100%[===================>]   1.54K  --.-KB/s    in 0s      

2022-09-27 07:04:40 (25.0 MB/s) - ‘data/download.py’ saved [1572/1572]

--2022-09-27 07:04:40--  https:/

In [10]:
if args.reload_from_files and os.path.exists(args.vectorizer_file):
    # 체크포인트 로드
    dataset = SurnameDataset.load_dataset_and_load_vectorizer(args.surname_csv, 
                                                              args.vectorizer_file)
else:
    # 데이터셋과 Vectorizer를 만듦
    dataset = SurnameDataset.load_dataset_and_make_vectorizer(args.surname_csv)
    dataset.save_vectorizer(args.vectorizer_file)

vectorizer = dataset.get_vectorizer()

classifier = SurnameClassifier(embedding_size=args.char_embedding_size, 
                               num_embeddings=len(vectorizer.char_vocab),
                               num_classes=len(vectorizer.nationality_vocab),
                               rnn_hidden_size=args.rnn_hidden_size,
                               padding_idx=vectorizer.char_vocab.mask_index)

In [11]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}


def update_train_state(args, model, train_state): # 훈련 상태 업데이트
                                                  # param args: 메인 매개변수
                                                  # param model: 훈련할 모델
                                                  # param train_state : 훈련 상태를 담은 딕셔너리
                                                  # 반환값 : 새로운 훈련 상태
    # 적어도 한 번 모델을 저장
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # 성능이 향상되면 모델을 저장
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # 손실이 나빠지면
        if loss_t >= loss_tm1:
            # 조기 종료 단계 업데이트
            train_state['early_stopping_step'] += 1
        # 손실이 감소하면
        else:
            # 최상의 모델 저장
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # 조기 종료 단계 재설정
            train_state['early_stopping_step'] = 0

        # 조기 종료 여부 확인
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state


def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

In [12]:
classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
    
loss_func = nn.CrossEntropyLoss(dataset.class_weights)
optimizer = optim.Adam(classifier.parameters(), lr=args.learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                           mode='min', factor=0.5,
                                           patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm.notebook.tqdm(desc='training routine', 
                               total=args.num_epochs,
                               position=0)

dataset.set_split('train')
train_bar = tqdm.notebook.tqdm(desc='split=train',
                               total=dataset.get_num_batches(args.batch_size), 
                               position=1, 
                               leave=True)
dataset.set_split('val')
val_bar = tqdm.notebook.tqdm(desc='split=val',
                             total=dataset.get_num_batches(args.batch_size), 
                             position=1, 
                             leave=True)

try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index

        # 훈련 세트에 대한 순회

        # 훈련 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('train')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 훈련 과정은 5단계

            # --------------------------------------
            # 단계 1. 그레이디언트를 0으로 초기화
            optimizer.zero_grad()

            # 단계 2. 출력 계산
            y_pred = classifier(x_in=batch_dict['x_data'], 
                                x_lengths=batch_dict['x_length'])

            # 단계 3. 손실 계산
            loss = loss_func(y_pred, batch_dict['y_target'])
    
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # 단계 4. 손실을 사용해 그레이디언트 계산
            loss.backward()

            # 단계 5. 옵티마이저로 가중치 업데이트
            optimizer.step()
            # -----------------------------------------
            
            # 정확도 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # 진행 상태 막대 업데이트
            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # 검증 세트에 대한 순회

        # 검증 세트와 배치 제너레이터 준비, 손실과 정확도를 0으로 설정
        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                                           batch_size=args.batch_size, 
                                           device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # 단계 1. 출력 계산
            y_pred = classifier(x_in=batch_dict['x_data'], 
                                x_lengths=batch_dict['x_length'])

            # 단계 2. 손실 계산
            loss = loss_func(y_pred, batch_dict['y_target'])
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # 단계 3. 정확도 계산
            acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)
            val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier, 
                                         train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        if train_state['stop_early']:
            break
            
except KeyboardInterrupt:
    print("반복 중지")

training routine:   0%|          | 0/100 [00:00<?, ?it/s]

split=train:   0%|          | 0/120 [00:00<?, ?it/s]

split=val:   0%|          | 0/25 [00:00<?, ?it/s]

In [13]:
# 가장 좋은 모델을 사용해 테스트 세트의 손실과 정확도를 계산
classifier.load_state_dict(torch.load(train_state['model_filename']))

classifier = classifier.to(args.device)
dataset.class_weights = dataset.class_weights.to(args.device)
loss_func = nn.CrossEntropyLoss(dataset.class_weights)

dataset.set_split('test')
batch_generator = generate_batches(dataset, 
                                   batch_size=args.batch_size, 
                                   device=args.device)
running_loss = 0.
running_acc = 0.
classifier.eval()

for batch_index, batch_dict in enumerate(batch_generator):
    # 출력 계산
    y_pred =  classifier(batch_dict['x_data'],
                         x_lengths=batch_dict['x_length'])
    
    # 손실 계산
    loss = loss_func(y_pred, batch_dict['y_target'])
    loss_t = loss.item()
    running_loss += (loss_t - running_loss) / (batch_index + 1)

    # 정확도 계산
    acc_t = compute_accuracy(y_pred, batch_dict['y_target'])
    running_acc += (acc_t - running_acc) / (batch_index + 1)

train_state['test_loss'] = running_loss
train_state['test_acc'] = running_acc

In [14]:
print("테스트 손실: {};".format(train_state['test_loss']))
print("테스트 정확도: {}".format(train_state['test_acc']))

테스트 손실: 1.8372113943099972;
테스트 정확도: 42.50000000000001


In [15]:
def predict_nationality(surname, classifier, vectorizer):
    vectorized_surname, vec_length = vectorizer.vectorize(surname)
    vectorized_surname = torch.tensor(vectorized_surname).unsqueeze(dim=0)
    vec_length = torch.tensor([vec_length], dtype=torch.int64)
    
    result = classifier(vectorized_surname, vec_length, apply_softmax=True)
    probability_values, indices = result.max(dim=1)
    
    index = indices.item()
    prob_value = probability_values.item()

    predicted_nationality = vectorizer.nationality_vocab.lookup_index(index)

    return {'nationality': predicted_nationality, 'probability': prob_value, 'surname': surname}

In [16]:
# surname = input("Enter a surname: ")
classifier = classifier.to("cpu")
for surname in ['McMahan', 'Nakamoto', 'Wan', 'Cho']:
    print(predict_nationality(surname, classifier, vectorizer))


{'nationality': 'Irish', 'probability': 0.30483436584472656, 'surname': 'McMahan'}
{'nationality': 'Japanese', 'probability': 0.7194134593009949, 'surname': 'Nakamoto'}
{'nationality': 'Vietnamese', 'probability': 0.42514726519584656, 'surname': 'Wan'}
{'nationality': 'Chinese', 'probability': 0.35451963543891907, 'surname': 'Cho'}
